## Rohan Gore 
###### rmg9725@nyu.edu
###### Collaborators: xAI - Grok
#

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, date_format, hour, count, rank, concat_ws, when, collect_list, struct, explode
from pyspark.sql.functions import lower, regexp_replace, split, rank, lit, udf, count, max as max_, min as min_
from pyspark.sql.window import Window
from pyspark.sql.types import ArrayType, StructType, StructField, StringType



spark = SparkSession.builder.appName("BakeryAnalysisQ1").getOrCreate()

df = spark.read.csv("shared/hw2/Bakery.csv", header=True, inferSchema=True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/22 00:33:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# df.head(200)

# Question 1

In [4]:
from pyspark.sql.functions import date_format
df = df.filter(col("Item") != "NONE")  
df_with_day = df.withColumn("DayOfWeek", date_format(col("Date"), "EEEE"))

In [5]:
from pyspark.sql.functions import hour
df_filtered = df_with_day.withColumn("Hour", hour(col("Time"))) \
                         .filter((col("Hour") >= 6) & (col("Hour") < 10))

In [6]:
from pyspark.sql.functions import count
grouped_df = df_filtered.groupBy("DayOfWeek", "Item") \
                        .agg(count("*").alias("total_qty"))

In [7]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank
window_spec = Window.partitionBy("DayOfWeek").orderBy(col("total_qty").desc())
ranked_df = grouped_df.withColumn("rank", rank().over(window_spec))


In [8]:
# top10_per_day = ranked_df.filter(col("rank") <= 10)
# final_df = top10_per_day.select(
#     col("Item"),
#     col("DayOfWeek").alias("day"),
#     col("total_qty").alias("qty")
# ).orderBy("day", col("total_qty").desc())

In [9]:
# top10_per_day = ranked_df.filter(col("rank") <= 10) \
#                          .select(
#                              col("Item"),
#                              col("DayOfWeek").alias("day"),
#                              col("total_qty").alias("qty")
#                          ) \
#                          .orderBy("day", col("total_qty").desc())


# formatted_df = top10_per_day.select(
#     concat_ws(", ", col("Item"), col("day"), col("qty")).alias("result")
# )

# # Collect and print each row
# for row in formatted_df.collect():
#     print(row["result"])

In [10]:
from pyspark.sql.functions import when

top10_per_day = ranked_df.filter(col("rank") <= 10) \
                         .select(
                             col("Item"),
                             col("DayOfWeek").alias("day"),
                             col("total_qty").alias("qty")
                         )

day_order = {
    "Monday": 1, "Tuesday": 2, "Wednesday": 3, "Thursday": 4,
    "Friday": 5, "Saturday": 6, "Sunday": 7
}


top10_with_order = top10_per_day.withColumn(
    "day_order",when(col("day") == "Monday", 1)
    .when(col("day") == "Tuesday", 2)
    .when(col("day") == "Wednesday", 3)
    .when(col("day") == "Thursday", 4)
    .when(col("day") == "Friday", 5)
    .when(col("day") == "Saturday", 6)
    .when(col("day") == "Sunday", 7)
    .otherwise(8) 
)

# Sorting by day_order and then total_qty descending
sorted_df = top10_with_order.orderBy("day_order", col("qty").desc()) \
                            .select("Item", "day", "qty")

formatted_df = sorted_df.select(
    concat_ws(", ", col("Item"), col("day"), col("qty")).alias("result")
)

for row in formatted_df.collect():
    print(row["result"])


Coffee, Monday, 108
Bread, Monday, 61
Pastry, Monday, 29
Medialuna, Monday, 19
Tea, Monday, 18
Farm House, Monday, 9
Toast, Monday, 7
Hot chocolate, Monday, 6
Juice, Monday, 5
Alfajores, Monday, 4
Coffee, Tuesday, 97
Bread, Tuesday, 58
Pastry, Tuesday, 35
Tea, Tuesday, 25
Medialuna, Tuesday, 12
Toast, Tuesday, 11
Farm House, Tuesday, 9
Cookies, Tuesday, 8
Hot chocolate, Tuesday, 8
Coke, Tuesday, 6
Coffee, Wednesday, 101
Bread, Wednesday, 78
Pastry, Wednesday, 35
Tea, Wednesday, 15
Medialuna, Wednesday, 13
Hot chocolate, Wednesday, 12
Toast, Wednesday, 10
Farm House, Wednesday, 6
Alfajores, Wednesday, 5
Jam, Wednesday, 5
Cake, Wednesday, 5
Coffee, Thursday, 109
Bread, Thursday, 84
Pastry, Thursday, 41
Medialuna, Thursday, 24
Tea, Thursday, 15
Toast, Thursday, 15
Hot chocolate, Thursday, 12
Farm House, Thursday, 12
Cookies, Thursday, 10
Cake, Thursday, 6
Coffee, Friday, 178
Bread, Friday, 118
Pastry, Friday, 42
Medialuna, Friday, 33
Toast, Friday, 28
Tea, Friday, 27
Cookies, Friday, 14
F

# Question 2

In [11]:
spark = SparkSession.builder.appName("BakeryAnalysisQ2").getOrCreate()
df = spark.read.csv("shared/hw2/Bakery.csv", header=True, inferSchema=True)
df_cleaned = df.filter(col("Item") != "NONE")

25/03/22 00:34:23 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


## Defined Daypart1 with 4 Elements:
* Morning --> (06:00 - 12:00)
* Afternoon --> (12:00 - 18:00)
* Evening --> (18:00 - 00:00)
* Night --> (00:00 - 06:00)

In [12]:
df_with_day = df_cleaned.withColumn("DayOfWeek", date_format(col("Date"), "EEEE")) \
                        .withColumn("Hour", hour(col("Time"))) \
                        .withColumn(
                            "Daypart2",
                            when(col("DayOfWeek").isin("Saturday", "Sunday"), "Weekend")
                            .otherwise("Weekday")
                        )

df_with_dayparts = df_with_day.withColumn(
    "Daypart1",
    when((col("Hour") >= 6) & (col("Hour") < 12), "Morning")
    .when((col("Hour") >= 12) & (col("Hour") < 18), "Afternoon")
    .when((col("Hour") >= 18) & (col("Hour") < 21), "Evening")
    .otherwise("Night")  # 21:00 to 05:59
)


# IMPORTANT --> Group by Transaction and Dayparts to collect items
items_per_tx = df_with_dayparts.groupBy("Transaction", "Daypart1", "Daypart2") \
                               .agg(collect_list("Item").alias("items"))


In [13]:

# UDF to generate pairs from a list of items
def generate_pairs(items):
    items = sorted(list(set(items)))  # Remove duplicates and sort for consistency
    pairs = [(items[i], items[j]) for i in range(len(items)) for j in range(i + 1, len(items))]
    return pairs

pair_schema = ArrayType(StructType([
    StructField("item1", StringType()),
    StructField("item2", StringType())
]))
generate_pairs_udf = udf(generate_pairs, pair_schema)


In [14]:
pairs_df = items_per_tx.withColumn("pairs", generate_pairs_udf(col("items"))) \
                       .select("Daypart1", "Daypart2", explode("pairs").alias("pair")) \
                       .select("Daypart1", "Daypart2", col("pair.item1"), col("pair.item2"))




pair_counts = pairs_df.groupBy("Daypart1", "Daypart2", "item1", "item2") \
                      .agg(count("*").alias("pair_count"))


window_spec = Window.partitionBy("Daypart1", "Daypart2").orderBy(col("pair_count").desc())


ranked_pairs = pair_counts.withColumn("rank", rank().over(window_spec))


In [15]:


top_pairs = ranked_pairs.filter(col("rank") == 1) \
                        .select("Daypart1", "Daypart2", "item1", "item2")


# Creating a DataFrame with all 8 Daypart combinations
all_dayparts = spark.createDataFrame([
    ("Morning", "Weekend"), ("Afternoon", "Weekend"), ("Evening", "Weekend"), ("Night", "Weekend"),
    ("Morning", "Weekday"), ("Afternoon", "Weekday"), ("Evening", "Weekday"), ("Night", "Weekday")
], ["Daypart1", "Daypart2"])



full_pairs = all_dayparts.join(top_pairs, ["Daypart1", "Daypart2"], "left_outer") \
                         .na.fill({"item1": "no pair", "item2": ""})



In [16]:

formatted_df = full_pairs.select(
    concat_ws(", ",
              col("Daypart1"),
              when(col("item1") == "no pair", lit("(no pair)"))
              .otherwise(concat_ws("", lit("("), col("item1"), lit(", "), col("item2"), lit(")"))),
              col("Daypart2")
    ).alias("result")
)



for row in formatted_df.orderBy(
    when(col("Daypart1") == "Morning", 1)
    .when(col("Daypart1") == "Afternoon", 2)
    .when(col("Daypart1") == "Evening", 3)
    .when(col("Daypart1") == "Night", 4),
    when(col("Daypart2") == "Weekend", 1)
    .when(col("Daypart2") == "Weekday", 2)
).collect():
    print(row["result"])


Morning, (Bread, Coffee), Weekend
Morning, (Bread, Coffee), Weekday
Afternoon, (Bread, Coffee), Weekend
Afternoon, (Bread, Coffee), Weekday
Evening, (Postcard, Tshirt), Weekend
Evening, (Coffee, Medialuna), Weekday
Evening, (Alfajores, Coffee), Weekday
Evening, (Coffee, Tea), Weekday
Night, (no pair), Weekend
Night, (Juice, Mineral water), Weekday


### The above output shows 3 entries for top pairs for Evening & Weekday
* This is because there are 3 pairs with same frequency count in that given daypart frame
* this can also be proved by executing the below code block
  
   * we can assess the output and find out that we have 3 pairs with "**Frequency == 2**"
 
   * Evening, (Alfajores, Coffee), Weekday, 2 
   * Evening, (Coffee, Medialuna), Weekday, 2
   * Evening, (Coffee, Tea), Weekday, 2

In [17]:
spark = SparkSession.builder.appName("BakeryPairCounts").getOrCreate()

df = spark.read.csv("shared/hw2/Bakery.csv", header=True, inferSchema=True)
df_cleaned = df.filter(col("Item") != "NONE")

df_with_day = df_cleaned.withColumn("DayOfWeek", date_format(col("Date"), "EEEE")) \
                        .withColumn("Hour", hour(col("Time"))) \
                        .withColumn(
                            "Daypart2",
                            when(col("DayOfWeek").isin("Saturday", "Sunday"), "Weekend")
                            .otherwise("Weekday")
                        )

df_with_dayparts = df_with_day.withColumn(
    "Daypart1",
    when((col("Hour") >= 6) & (col("Hour") < 12), "Morning")
    .when((col("Hour") >= 12) & (col("Hour") < 18), "Afternoon")
    .when((col("Hour") >= 18) & (col("Hour") < 21), "Evening")
    .otherwise("Night")
)

items_per_tx = df_with_dayparts.groupBy("Transaction", "Daypart1", "Daypart2") \
                               .agg(collect_list("Item").alias("items"))

def generate_pairs(items):
    items = sorted(list(set(items)))
    pairs = [(items[i], items[j]) for i in range(len(items)) for j in range(i + 1, len(items))]
    return pairs


pair_schema = ArrayType(StructType([
    StructField("item1", StringType()),
    StructField("item2", StringType())
]))
generate_pairs_udf = udf(generate_pairs, pair_schema)


pairs_df = items_per_tx.withColumn("pairs", generate_pairs_udf(col("items"))) \
                       .select("Daypart1", "Daypart2", explode("pairs").alias("pair")) \
                       .select("Daypart1", "Daypart2", col("pair.item1"), col("pair.item2"))

pair_counts = pairs_df.groupBy("Daypart1", "Daypart2", "item1", "item2") \
                      .agg(count("*").alias("pair_count"))

formatted_counts = pair_counts.select(
    concat_ws(", ",
              col("Daypart1"),
              concat_ws("", lit("("), col("item1"), lit(", "), col("item2"), lit(")")),
              col("Daypart2"),
              col("pair_count")
    ).alias("result")
)

print("All pairs with counts (Daypart1, (item1, item2), Daypart2, count):")
for row in formatted_counts.orderBy(
    when(col("Daypart1") == "Morning", 1)
    .when(col("Daypart1") == "Afternoon", 2)
    .when(col("Daypart1") == "Evening", 3)
    .when(col("Daypart1") == "Night", 4),
    when(col("Daypart2") == "Weekend", 1)
    .when(col("Daypart2") == "Weekday", 2),
    col("pair_count").desc(),
    col("item1").asc(),
    col("item2").asc()
).collect():
    print(row["result"])


25/03/22 00:34:41 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


All pairs with counts (Daypart1, (item1, item2), Daypart2, count):


Morning, (Bread, Coffee), Weekend, 149
Morning, (Coffee, Medialuna), Weekend, 110
Morning, (Coffee, Pastry), Weekend, 99
Morning, (Bread, Pastry), Weekend, 79
Morning, (Bread, Medialuna), Weekend, 64
Morning, (Coffee, Hot chocolate), Weekend, 61
Morning, (Coffee, Tea), Weekend, 57
Morning, (Coffee, Scone), Weekend, 48
Morning, (Cake, Coffee), Weekend, 47
Morning, (Coffee, Toast), Weekend, 40
Morning, (Brownie, Coffee), Weekend, 38
Morning, (Coffee, Juice), Weekend, 34
Morning, (Coffee, Cookies), Weekend, 33
Morning, (Medialuna, Pastry), Weekend, 33
Morning, (Coffee, Muffin), Weekend, 32
Morning, (Bread, Hot chocolate), Weekend, 30
Morning, (Coffee, Spanish Brunch), Weekend, 29
Morning, (Bread, Scone), Weekend, 26
Morning, (Bread, Cake), Weekend, 25
Morning, (Bread, Tea), Weekend, 23
Morning, (Coffee, Farm House), Weekend, 21
Morning, (Bread, Brownie), Weekend, 20
Morning, (Bread, Farm House), Weekend, 20
Morning, (Bread, Toast), Weekend, 20
Morning, (Hot chocolate, Pastry), Weekend, 20

# Question 3

In [18]:
spark = SparkSession.builder.appName("PopulationAnalysisQ3").getOrCreate()
spark.sparkContext.setLogLevel("ERROR") 

df = spark.read.csv("shared/hw2/populationbycountry19802010millions.csv", header=True, inferSchema=True)
df.cache() 

25/03/22 00:34:49 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


DataFrame[_c0: string, 1980: string, 1981: string, 1982: string, 1983: string, 1984: string, 1985: string, 1986: string, 1987: string, 1988: string, 1989: string, 1990: string, 1991: string, 1992: string, 1993: string, 1994: string, 1995: string, 1996: string, 1997: string, 1998: string, 1999: string, 2000: string, 2001: string, 2002: string, 2003: string, 2004: string, 2005: string, 2006: string, 2007: string, 2008: string, 2009: string, 2010: string]

In [19]:
year_cols = [str(y) for y in range(1980, 2011)]
cast_exprs = [col(c).cast("float").alias(c) for c in year_cols]


df = df.withColumnRenamed("_c0", "Country").select("Country", *cast_exprs)



regions_to_exclude = [
    "North America", "South America", "Africa", "Asia", "Europe", "Oceania", "World",
    "Latin America and the Caribbean", "Western Asia", "Sub-Saharan Africa"
]



df_countries = df.filter(~col("Country").isin(regions_to_exclude))

In [20]:

increase_exprs = []


for i in range(1, len(year_cols)):  # 1981 to 2010
    curr_year = year_cols[i]
    prev_year = year_cols[i-1]

    increase_col = (
        ((col(curr_year) - col(prev_year)) / col(prev_year) * 100)
        .alias(f"{curr_year}_pct_increase")
    )
    
    increase_exprs.append(increase_col)


df_increases = df_countries.select("Country", *increase_exprs)

increase_cols = [f"{y}_pct_increase" for y in range(1981, 2011)]
rows = []


for year_col in increase_cols:
    year = year_col.split("_")[0]
    rows.append(
        df_increases.select(
            lit(year).alias("Year"),
            col("Country"),
            col(year_col).alias("pct_increase")
        )
    )

In [21]:

df_long = rows[0]

for r in rows[1:]:
    df_long = df_long.union(r)

df_long = df_long.filter(col("pct_increase").isNotNull())

# Finding the country with max increase per year
window_spec = Window.partitionBy("Year").orderBy(col("pct_increase").desc())
yearly_tops = df_long.withColumn("rank", rank().over(window_spec)) \
                     .filter(col("rank") == 1) \
                     .select("Year", "Country", "pct_increase")





In [22]:
print("Yearly top percentage increases:")
yearly_results = yearly_tops.orderBy("Year").collect()
for row in yearly_results:
    print(f"{row['Year']}, {row['Country']}, {row['pct_increase']:.2f}%")

Yearly top percentage increases:


1981, Western Sahara, 12.13%
1982, Western Sahara, 11.12%
1983, French Guiana, 14.29%
1984, Qatar, 10.96%
1985, French Guiana, 12.50%
1986, Qatar, 8.77%
1987, French Guiana, 11.11%
1988, Cayman Islands, 11.01%
1989, United Arab Emirates, 6.12%
1990, Djibouti, 12.82%
1991, Jordan, 11.27%
1992, Kuwait, 48.63%
1993, Afghanistan, 13.22%
1994, Afghanistan, 8.73%
1995, Burundi, 7.22%
1996, Rwanda, 19.61%
1997, Falkland Islands (Islas Malvinas), 21.50%
1998, Liberia, 12.02%
1999, Falkland Islands (Islas Malvinas), 7.69%
2000, Montserrat, 16.86%
2001, Montserrat, 7.34%
2002, Montserrat, 13.44%
2003, Afghanistan, 5.80%
2004, Montserrat, 10.47%
2005, Liberia, 4.80%
2006, Jordan, 7.09%
2007, Jordan, 6.76%
2008, Montserrat, 12.64%
2009, Liberia, 4.16%
2010, Niger, 3.74%


In [23]:
overall_top = yearly_tops.orderBy(col("pct_increase").desc()).limit(1).collect()[0]
overall_lowest = yearly_tops.orderBy(col("pct_increase").asc()).limit(1).collect()[0]

print("\nOverall results:")
print(f"{overall_top['Year']}, {overall_top['Country']} (biggest rate increase: {overall_top['pct_increase']:.2f}%)")
print(f"{overall_lowest['Year']}, {overall_lowest['Country']} (lowest increase/decrease: {overall_lowest['pct_increase']:.2f}%)")


Overall results:
1992, Kuwait (biggest rate increase: 48.63%)
2010, Niger (lowest increase/decrease: 3.74%)


# Question 4

In [24]:
spark = SparkSession.builder.appName("WordCountQ4").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")  # Suppress warnings

df = spark.read.text("shared/hw2/hw1dir")  # Reads all files in directory

words_df = df.select(
    explode(
        split(
            regexp_replace(lower(col("value")), "[^0-9a-z]", " "),
            "\\s+"
        )
    ).alias("word")
).filter(col("word") != "") 

In [25]:
word_counts = words_df.groupBy("word").count() \
                      .orderBy(col("count").desc()) \
                      .limit(20)

print("Top 20 words:")
word_counts.show(truncate=False)

Top 20 words:


+----+------+
|word|count |
+----+------+
|the |163547|
|to  |89046 |
|p   |78664 |
|of  |75568 |
|and |72730 |
|in  |56782 |
|a   |53198 |
|for |29770 |
|that|28852 |
|is  |27601 |
|on  |24485 |
|s   |23615 |
|with|19575 |
|are |19417 |
|it  |18231 |
|be  |17998 |
|as  |17796 |
|have|16188 |
|at  |15965 |
|we  |15754 |
+----+------+



# Question 5

#### Without SparkML

In [26]:
from collections import Counter

file_path = "shared/hw2/internet_archive_scifi_v3.txt"
trigram_counter = Counter()

try:
    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            words = line.strip().split()
            if len(words) >= 3:
                trigrams = [(words[i], words[i+1], words[i+2]) for i in range(len(words) - 2)]
                trigram_counter.update(trigrams)
except FileNotFoundError:
    print(f"File not found: {file_path}")
    raise

top_10 = trigram_counter.most_common(10)

print("Top 10 most common trigrams:")
for trigram, count in top_10:
    print(f"{' '.join(trigram)}, {count}")

Top 10 most common trigrams:
one of the, 9081
out of the, 8135
There was a, 4674
It was a, 4254
back to the, 3857
a lot of, 3606
to be a, 3446
be able to, 3399
the rest of, 3358
there was a, 3204


#### With SparkML (does not work)

In [ ]:
from pyspark.ml.feature import Tokenizer, NGram
from pyspark.sql.functions import explode, col, count
from pyspark.storagelevel import StorageLevel

try:
    spark = (
        SparkSession.builder
        .appName("SciFiTrigrams")
        .config("spark.driver.host", "localhost")
        .config("spark.driver.memory", "8g")
        .config("spark.executor.memory", "4g")
        .config("spark.sql.shuffle.partitions", "200")
        .getOrCreate()
    )
    
    # Load with explicit partitioning
    df = spark.read.text("shared/hw2/internet_archive_scifi_v3.txt", 
                        wholetext=False, lineSep="\n")
    
    # Pipeline with checkpointing
    tokenizer = Tokenizer(inputCol="value", outputCol="words")
    words_df = tokenizer.transform(df)
    
    trigram = NGram(n=3, inputCol="words", outputCol="trigrams")
    trigrams_df = trigram.transform(words_df).persist(StorageLevel.MEMORY_AND_DISK)
    
    exploded_df = trigrams_df.select(explode("trigrams").alias("trigram"))
    trigram_counts = exploded_df.groupBy("trigram").agg(count("trigram").alias("count"))
    
    # Distributed write instead of .show()
    (trigram_counts.orderBy(col("count").desc())
     .limit(10)
     .write.mode("overwrite")
     .csv("results/top_trigrams"))
    
except Exception as e:
    print(f"Pipeline failed: {str(e)}")
    raise
    
finally:
    spark.stop()
    !jupyter kernel restart  # Force clean environment


In [27]:
print ("GG")

GG
